In [15]:
import urllib.request

In [16]:
url= 'http://zims-en.kiwix.campusafrica.gos.orange.com/wikipedia_en_all_nopic/A/List_of_postal_codes_of_Canada:_M'

In [17]:
html_doc=urllib.request.urlopen(url)

In [18]:

from bs4 import BeautifulSoup

In [19]:
b_soup=BeautifulSoup(html_doc)

In [20]:
print(b_soup.prettify())

<!DOCTYPE html>
<html class="client-js">
 <head>
  <script src="/skin/block_external.js" type="text/javascript">
  </script>
  <link href="/skin/jquery-ui/jquery-ui.min.css" rel="Stylesheet" type="text/css"/>
  <link href="/skin/jquery-ui/jquery-ui.theme.min.css" rel="Stylesheet" type="text/css"/>
  <link href="/skin/taskbar.css" rel="Stylesheet" type="text/css"/>
  <script src="/skin/jquery-ui/external/jquery/jquery.js" type="text/javascript">
  </script>
  <script src="/skin/jquery-ui/jquery-ui.min.js" type="text/javascript">
  </script>
  <script>
   var jk = jQuery.noConflict();
  jk(function() {
  jk( "#kiwixsearchbox" ).autocomplete({

  source: "/suggest?content=wikipedia_en_all_nopic",
  dataType: "json",
  cache: false,

  select: function(event, ui) {
  jk( "#kiwixsearchbox" ).val(ui.item.value);
  jk( "#kiwixsearchform" ).submit();
  },

  });
  });

  /* cybook hack */
  if (navigator.userAgent.indexOf("bookeen/cybook") != -1) {
  	jk("html").addClass("cybook");
  }
  </scr

In [21]:

import pandas as pd
import numpy as np

In [23]:
columns=[]
for column in b_soup.find_all('th'):
    columns.append(column.get_text())

In [24]:
columns

['Postcode', 'Borough', 'Neighbourhood']

In [25]:
columns[0]='PostalCode'
columns[2]='Neighborhood'

In [26]:
columns

['PostalCode', 'Borough', 'Neighborhood']

In [27]:
df=pd.DataFrame(columns=columns)
df

,PostalCode,Borough,Neighborhood


In [29]:
A=[]
B=[]
C=[]
for row in b_soup.find_all("tr"):
    elements=row.find_all('td')
    if len(elements)==3:
        A.append(elements[0].get_text().strip())
        B.append(elements[1].get_text().strip())
        C.append(elements[2].get_text().strip())

In [30]:
df['PostalCode']=A
df['Borough']=B
df['Neighborhood']=C

In [31]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [32]:
df.replace('Not assigned',np.nan,inplace=True)

In [33]:
df

,PostalCode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


In [34]:
df.dropna(subset=['Borough'],axis=0,inplace=True)

In [35]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [36]:
df.reset_index(drop=True,inplace=True)

In [37]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [38]:
df['Neighborhood']=df['Neighborhood'].replace(np.nan,df['Borough'])

In [39]:
df=df.astype(str)

In [40]:
df=df.groupby(df['PostalCode'],sort=False).agg(lambda x: ', '.join(set(x))).reset_index()

In [41]:

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


In [42]:
df.shape


(103, 3)

In [43]:
df.to_csv('TorontoDataset.csv')